In [1]:
import numpy as np
from data import X_train, X_test, y_train, y_test
from tree import print_tree
from random_forest import RF


In [60]:
random_forest = RF(n_estimators=100, max_depth=5, min_samples_split=5, min_samples_leaf=2, max_features=5, bootstrap=True) 
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
acc = np.mean(y_pred == y_test)
print(f"Test accuracy: {acc}")


Test accuracy: 0.9824561403508771


In [5]:
def k_fold_cross_validation(X, y, k=5, max_depth=None, min_samples_split=2, min_samples_leaf=1, seed=40,n_estimators=100,max_features=30,bootstrap=False):
    n_samples = len(y)
    indices = np.arange(n_samples)
    np.random.seed(seed)
    np.random.shuffle(indices)  # Mezclar datos

    fold_sizes = np.full(k, n_samples // k, dtype=int)
    fold_sizes[: n_samples % k] += 1  # Repartir sobrantes
    current = 0

    scores_val = []
    scores_train = []

    for fold_size in fold_sizes:
        start, stop = current, current + fold_size
        val_idx = indices[start:stop]
        train_idx = np.concatenate((indices[:start], indices[stop:]))

        X_train_fold, y_train_fold = X[train_idx], y[train_idx]
        X_val_fold, y_val_fold = X[val_idx], y[val_idx]


        random_forest = RF(n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features, bootstrap)
        random_forest.fit(X_train, y_train)


        y_pred_train= random_forest.predict(X_train_fold)
        acc_train = np.mean(y_pred_train == y_train_fold)

        y_pred_val = random_forest.predict(X_val_fold)
        acc_val = np.mean(y_pred_val == y_val_fold)
        # Predecir en entrenamiento y validación
        scores_val.append(acc_val)
        scores_train.append(acc_train)

        current = stop

    train = np.mean(scores_train)
    val = np.mean(scores_val)
    desvtrain = np.std(scores_train)
    desvval = np.std(scores_val)

    return train, val, desvtrain, desvval

In [62]:
score_train, score_val, desv_train, desv_val = k_fold_cross_validation(X_train, y_train, k=5, max_depth=5, min_samples_split=5, min_samples_leaf=2, seed=40,n_estimators=100,max_features=30,bootstrap=True)
print(f"Train accuracy: {score_train} ± {desv_train}")
print(f"Validation accuracy: {score_val} ± {desv_val}")

Train accuracy: 0.9857142857142858 ± 0.004037620455137144
Validation accuracy: 0.9824175824175825 ± 0.011206636293610524
